In [ ]:
# 필요한 라이브러리 임포트
import os
import json
from dotenv import load_dotenv # .env 파일 로드를 위해 추가

from langchain_core.documents import Document
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, chain
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.output_parsers import StrOutputParser

from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.tools.tavily_search import TavilySearchResults # TavilySearchResults 유지
from langchain_community.utilities import WikipediaAPIWrapper # <--- WikipediaAPIWrapper로 수정!

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_openai import ChatOpenAI

# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수 확인 (선택 사항)
tavily_key = os.getenv("TAVILY_API_KEY")
if tavily_key:
    print(f"TAVILY_API_KEY가 성공적으로 로드되었습니다. (일부 가려짐: {tavily_key[:5]}...)")
else:
    print("경고: TAVILY_API_KEY가 로드되지 않았습니다. .env 파일을 확인해주세요.")

print("필요한 라이브러리 임포트 및 환경 변수 설정이 완료되었습니다.")

In [ ]:
# OllamaEmbeddings 인스턴스 생성 (DB 로드 및 구축 시 필요)
embeddings = OllamaEmbeddings(model="bge-m3")

# 벡터 DB 경로 설정
file_path = "../data/cafe_menu_data.txt"
db_path = "../db/cafe_db"

# DB 디렉토리가 없으면 생성
os.makedirs("./db", exist_ok=True)

# DB 파일이 존재하는지 확인 (재구축 방지)
if not os.path.exists(os.path.join(db_path, "index.faiss")):
    print("벡터 DB가 존재하지 않아 새로 구축합니다...")
    # cafe_menu.txt 파일 로드
    loader = TextLoader(file_path, encoding="utf-8")
    documents = loader.load()

    # 각 메뉴 항목을 별도의 문서로 분할
    split_documents = []
    current_doc_content = ""
    for line in documents[0].page_content.split('\n'):
        if line.strip() and not line.startswith(' '): # 메뉴 이름 (새로운 메뉴 시작)
            if current_doc_content:
                split_documents.append(Document(page_content=current_doc_content.strip()))
            current_doc_content = line + '\n'
        else:
            current_doc_content += line + '\n'
    if current_doc_content: # 마지막 메뉴 추가
        split_documents.append(Document(page_content=current_doc_content.strip()))

    # FAISS를 사용한 벡터 인덱스 생성 및 저장
    vectorstore = FAISS.from_documents(split_documents, embeddings)
    vectorstore.save_local(db_path)
    print(f"벡터 DB가 '{db_path}' 경로에 성공적으로 생성되었습니다.")
else:
    print(f"벡터 DB가 '{db_path}' 경로에 이미 존재합니다. 로드합니다.")

# 벡터 DB 로드
try:
    vectorstore = FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)
    print("벡터 DB 로드 완료.")
except Exception as e:
    print(f"벡터 DB 로드 중 오류 발생: {e}")
    vectorstore = None # 오류 발생 시 None으로 설정하여 이후 코드에서 처리

# 벡터 DB 테스트 (선택 사항)
if vectorstore:
    query = "아메리카노"
    docs = vectorstore.similarity_search(query)
    print(f"\n벡터 DB 테스트 결과 ('{query}' 검색):")
    for doc in docs:
        print(doc.page_content[:100] + "...") # 내용의 일부만 출력하여 길이를 제한
else:
    print("벡터 DB가 로드되지 않아 테스트를 수행할 수 없습니다.")

In [ ]:
# 전역 변수 'vectorstore'가 셀 2에서 로드되었는지 확인
if 'vectorstore' not in locals() or vectorstore is None:
    print("경고: 'vectorstore'가 로드되지 않았습니다. 셀 2를 먼저 실행해주세요.")
    vectorstore = None

# a) tavily_search_func 도구 정의
@tool
def tavily_search_func(query: str) -> str:
    """웹에서 최신 정보를 검색합니다. 최신 커피 트렌드, 카페 위치 정보 등을 찾을 때 유용합니다."""
    if not os.getenv("TAVILY_API_KEY"):
        return "TAVILY_API_KEY가 설정되지 않았습니다. 웹 검색 기능을 사용할 수 없습니다. .env 파일을 확인해주세요."
    tavily = TavilySearchResults(max_results=3)
    return tavily.run(query)

# b) wiki_summary 도구 정의
@tool
def wiki_summary(query: str) -> str:
    """위키피디아에서 일반 지식을 검색하고 요약합니다. 커피 역사, 음료 제조 방법 등을 찾을 때 유용합니다."""
    # WikipediaAPIWrapper를 사용하여 Wikipedia 검색 기능 제공
    wiki_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200) # 검색 결과 1개, 최대 200자로 제한
    try:
        # WikipediaAPIWrapper의 run 메서드는 문자열 쿼리를 직접 받습니다.
        return wiki_wrapper.run(query) # <--- 이 부분 수정!
    except Exception as e:
        return f"위키피디아 검색 중 오류 발생: {e}. 해당 주제에 대한 정보를 찾을 수 없을 수 있습니다."

# c) db_search_cafe_func 도구 정의
@tool
def db_search_cafe_func(query: str) -> str:
    """로컬 카페 메뉴 DB에서 정보를 검색합니다. 특정 메뉴의 가격, 재료, 설명 등을 찾을 때 유용합니다."""
    if vectorstore is None:
        return "카페 메뉴 DB가 로드되지 않았습니다. DB 구축 단계를 먼저 실행해주세요."
    docs = vectorstore.similarity_search(query, k=1)
    if docs:
        return docs[0].page_content
    return "요청하신 메뉴 정보를 찾을 수 없습니다."

# LLM 정의 (Ollama의 Llama2 모델 사용 예시)
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.3  
)


# LLM에 정의된 도구들을 바인딩
tools = [tavily_search_func, wiki_summary, db_search_cafe_func]
llm_with_tools = llm.bind_tools(tools)

print("3가지 도구(tavily_search_func, wiki_summary, db_search_cafe_func)가 정의되었고,")
print("LLM에 성공적으로 바인딩되었습니다.")

In [ ]:
# 도구 이름과 도구 함수의 매핑
tool_map = {tool.name: tool for tool in tools}

# 도구 실행 함수 정의
def call_tool(tool_invocation: dict):
    """LangChain ToolInvocation 객체를 받아 실제 도구를 실행합니다."""
    tool_name = tool_invocation["name"]
    tool_args = tool_invocation["args"]
    # print(f"DEBUG: Calling tool: {tool_name} with args: {tool_args}") # 디버깅용
    tool_func = tool_map.get(tool_name)
    if tool_func:
        return tool_func.invoke(tool_args)
    else:
        return f"오류: 알 수 없는 도구 '{tool_name}' 입니다."

# 도구 호출 결과를 처리하는 체인
@chain
def tool_calling_chain(messages):
    """
    LLM의 응답에서 도구 호출을 감지하고, 해당 도구를 실행한 후 결과를 LLM에 다시 전달합니다.
    """
    response = llm_with_tools.invoke(messages)
    # print(f"DEBUG: LLM Response: {response}") # 디버깅용

    # LLM이 도구 호출을 제안했는지 확인
    if not response.tool_calls:
        # 도구 호출이 없으면, LLM의 최종 메시지를 반환
        return response.content

    # 도구 호출이 있으면, 각 도구를 실행하고 결과를 메시지로 변환
    tool_output_messages = []
    for tool_call in response.tool_calls:
        tool_output = call_tool(tool_call)
        tool_output_messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

    # 도구 호출 메시지를 원래 메시지에 추가하여 다시 LLM에 전달
    return llm.invoke(messages + tool_output_messages).content

print("도구 호출 체인 (tool_calling_chain) 구현 완료.")

In [ ]:
# 테스트 질문
question = "아메리카노의 가격과 특징은 무엇인가요?"
print(f"질문: {question}\n")

# 체인 실행
try:
    # HumanMessage 형태로 질문을 전달
    result = tool_calling_chain.invoke([HumanMessage(content=question)])
    print("--- 최종 답변 ---")
    print(result)
except Exception as e:
    print(f"체인 실행 중 오류 발생: {e}")
    print("Ollama 서버가 실행 중인지, 필요한 모델이 pull 되어 있는지, API 키가 올바르게 설정되었는지 확인해주세요.")

print("\n\n--- 추가 테스트 ---")

# 추가 질문 1: 웹 검색이 필요한 질문
question_web = "2024년 최신 커피 트렌드는 무엇인가요?"
print(f"\n질문: {question_web}\n")
try:
    result_web = tool_calling_chain.invoke([HumanMessage(content=question_web)])
    print("--- 최종 답변 ---")
    print(result_web)
except Exception as e:
    print(f"체인 실행 중 오류 발생: {e}")

# 추가 질문 2: 위키피디아 검색이 필요한 질문
question_wiki = "에스프레소의 역사를 알려주세요."
print(f"\n질문: {question_wiki}\n")
try:
    result_wiki = tool_calling_chain.invoke([HumanMessage(content=question_wiki)])
    print("--- 최종 답변 ---")
    print(result_wiki)
except Exception as e:
    print(f"체인 실행 중 오류 발생: {e}")